In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

First step of data processing is always, loading the data then inspecting it.

In [2]:
data = pd.read_csv('./sample_customer_data_100_records.csv')
print(data.describe())
data.head(100) # Inspecting data to see missing data or invalid data

       CustomerID        Age  Purchase Amount
count  100.000000  84.000000        80.000000
mean    50.500000  38.392857       401.250000
std     29.011492   8.034591       327.048075
min      1.000000  25.000000       100.000000
25%     25.750000  30.000000       175.000000
50%     50.500000  40.000000       300.000000
75%     75.250000  45.000000       500.000000
max    100.000000  50.000000      1000.000000


,CustomerID,Age,Gender,Product Category,Cart Added,Purchase Amount,Purchased,Satisfaction Level
0,1,NaN,Female,Electronics,No,NaN,No,Low
1,2,40.0,Female,Grocery,Yes,100.0,No,Low
2,3,45.0,Male,Grocery,Yes,300.0,Yes,High
3,4,NaN,Female,Electronics,No,100.0,Yes,High
4,5,35.0,Male,Furniture,Yes,1000.0,No,Low
...,...,...,...,...,...,...,...,...
95,96,50.0,Female,Clothing,No,200.0,No,Low
96,97,40.0,Female,Furniture,No,300.0,Yes,High
97,98,50.0,Male,Clothing,No,200.0,No,Low
98,99,NaN,Male,Furniture,Yes,100.0,Yes,High


According to the above observations, the age and purchase amount have missing values. Age data is not very skewed, so we may replace missing values with the mean. In contrast, Purchase amount is the opposite where it has some outliers. Therefore using the median to replace missing values would be the better option.

In [3]:
data['Age'] = data['Age'].fillna(round(data['Age'].mean(), 1))
data.columns = data.columns.str.strip() # because some columns had leading or trailing spaces
data['Purchase Amount'] = data['Purchase Amount'].fillna(round(data['Purchase Amount'].median(), 1))

In [4]:
# check if there are any duplicates
duplicates = data.duplicated()
print('Number of duplicates:', duplicates.sum())

# check for missing data
data.isnull().sum()

Number of duplicates: 0


CustomerID            0
Age                   0
Gender                0
Product Category      0
Cart Added            0
Purchase Amount       0
Purchased             0
Satisfaction Level    0
dtype: int64

No more missing values or duplicates, so we can go forward with encoding now.

In [5]:
data = pd.get_dummies(data, columns=["Gender", "Product Category"])

le = LabelEncoder()
data['Satisfaction Level'] = le.fit_transform(data['Satisfaction Level'])
data.head()


,CustomerID,Age,Cart Added,Purchase Amount,Purchased,Satisfaction Level,Gender_Female,Gender_Male,Product Category_Clothing,Product Category_Electronics,Product Category_Furniture,Product Category_Grocery
0,1,38.4,No,300.0,No,1,True,False,False,True,False,False
1,2,40.0,Yes,100.0,No,1,True,False,False,False,False,True
2,3,45.0,Yes,300.0,Yes,0,False,True,False,False,False,True
3,4,38.4,No,100.0,Yes,0,True,False,False,True,False,False
4,5,35.0,Yes,1000.0,No,1,False,True,False,False,True,False


All data has been encoded! next step from here is Data scaling & normalization. I use MinMaxScaler since it transforms the features by scaling them to a specified range between 0 and 1.

In [6]:
# Normalize the 'Purchase Amount' column to ensure all values are between 0 and 1
mms = MinMaxScaler()
data['Purchase Amount'] = mms.fit_transform(data[['Purchase Amount']])
data['Purchase Amount']

0     0.222222
1     0.000000
2     0.222222
3     0.000000
4     1.000000
        ...   
95    0.111111
96    0.222222
97    0.111111
98    0.000000
99    1.000000
Name: Purchase Amount, Length: 100, dtype: float64

Since all steps now are done, i write the new cleansed data to a new csv file, to not override the old one for back up purposes, in case any errors are made in future analytical steps.

In [7]:
# write back to csv
data.to_csv('cleaned_sample_customer_data_100_records.csv', index=False)